In [6]:
!pip install fabric
from fabric import Connection

460.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


In [86]:
con = Connection(host= 'agpbruger@130.225.170.33', port = 22022, connect_kwargs= {"password": "MY_SECRET_PASSWORD"})
result = con.run('uname -s', hide=True)
print(result.stdout)
con2 = Connection(host= 'agpbruger@130.225.170.33', port = 22001, connect_timeout=2, connect_kwargs= {"key_filename": "/home/chris/.ssh/id_ed25519"})
result2 = con2.run('uname -s', hide=True)
print(result2.stdout == result.stdout)

Linux

True


In [77]:
result = con.run('whoami', hide=True)
print(result.stdout)

agpbruger



In [83]:
with Connection('localhost',user="chris") as conn:
    conn.run('uname -s')
    conn.run('df -H')

Linux
Filesystem      Size  Used Avail Use% Mounted on
tmpfs           814M  2.0M  812M   1% /run
/dev/vda3        26G   19G  5.7G  77% /
tmpfs           4.1G     0  4.1G   0% /dev/shm
tmpfs           5.3M     0  5.3M   0% /run/lock
/dev/vda2       537M  5.5M  532M   2% /boot/efi
tmpfs           814M  4.9M  809M   1% /run/user/1000


In [49]:
from pprint import pprint
con_local = Connection('localhost',user="chris")
res = con_local.run('ps aux', hide=True)
pprint(res.stdout)

('USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND\n'
 'root           1  0.3  0.1 168092 13324 ?        Rs   07:55   1:18 '
 '/sbin/init splash\n'
 'root           2  0.0  0.0      0     0 ?        S    07:55   0:00 '
 '[kthreadd]\n'
 'root           3  0.0  0.0      0     0 ?        I<   07:55   0:00 [rcu_gp]\n'
 'root           4  0.0  0.0      0     0 ?        I<   07:55   0:00 '
 '[rcu_par_gp]\n'
 'root           5  0.0  0.0      0     0 ?        I<   07:55   0:00 [netns]\n'
 'root           7  0.0  0.0      0     0 ?        I<   07:55   0:00 '
 '[kworker/0:0H-events_highpri]\n'
 'root           9  0.0  0.0      0     0 ?        I<   07:55   0:00 '
 '[mm_percpu_wq]\n'
 'root          10  0.0  0.0      0     0 ?        S    07:55   0:00 '
 '[rcu_tasks_rude_]\n'
 'root          11  0.0  0.0      0     0 ?        S    07:55   0:00 '
 '[rcu_tasks_trace]\n'
 'root          12  0.0  0.0      0     0 ?        S    07:55   0:01 '
 '[ksoftirqd/0]\n'
 'root        

In [54]:
def process_list(c, process):
    res = c.run(f'ps aux | grep -i {process}' , hide=True)
    return res.stdout
pprint(process_list(con_local, '/etc/tor'))

('debian-+     836  0.0  0.6  59788 49844 ?        Ss   07:55   0:08 '
 '/usr/bin/tor --defaults-torrc /usr/share/tor/tor-service-defaults-torrc -f '
 '/etc/tor/torrc --RunAsDaemon 0\n'
 'chris      20475  0.0  0.0  18624  3640 ?        Ss   14:16   0:00 bash -c '
 'ps aux | grep -i /etc/tor\n'
 'chris      20480  0.0  0.0  17864  2376 ?        S    14:16   0:00 grep -i '
 '/etc/tor\n')


In [59]:
# Use a group of hosts to run a command
from fabric import SerialGroup
group = SerialGroup("agpbruger@130.225.170.33:22001","agpbruger@130.225.170.33:22002","agpbruger@130.225.170.33:22003","agpbruger@130.225.170.33:22004","agpbruger@130.225.170.33:22005","agpbruger@130.225.170.33:22006")
results = group.run('unameaasd -s',hide=True)

GroupException: {<Connection host=130.225.170.33 user=agpbruger port=22001>: <UnexpectedExit: cmd='unameaasd -s' exited=127>, <Connection host=130.225.170.33 user=agpbruger port=22002>: <UnexpectedExit: cmd='unameaasd -s' exited=127>, <Connection host=130.225.170.33 user=agpbruger port=22003>: <UnexpectedExit: cmd='unameaasd -s' exited=127>, <Connection host=130.225.170.33 user=agpbruger port=22004>: <UnexpectedExit: cmd='unameaasd -s' exited=127>, <Connection host=130.225.170.33 user=agpbruger port=22005>: <UnexpectedExit: cmd='unameaasd -s' exited=127>, <Connection host=130.225.170.33 user=agpbruger port=22006>: <UnexpectedExit: cmd='unameaasd -s' exited=127>}

In [60]:
# Work with a GroupResult
for connection, result in results.items():
     print("{0.host}:{1.port}: {2.stdout}".format(connection,connection, result))


130.225.170.33:22001: Linux

130.225.170.33:22002: Linux

130.225.170.33:22003: Linux

130.225.170.33:22004: Linux

130.225.170.33:22005: Linux

130.225.170.33:22006: Linux



In [95]:
# Define function and run it on a group of hosts
def free_space(g):
    return g.run('df -h',hide=True)
free_space(group)


{<Connection host=130.225.170.33 user=agpbruger port=22001>: <Result cmd='df -h' exited=0>,
 <Connection host=130.225.170.33 user=agpbruger port=22002>: <Result cmd='df -h' exited=0>,
 <Connection host=130.225.170.33 user=agpbruger port=22003>: <Result cmd='df -h' exited=0>,
 <Connection host=130.225.170.33 user=agpbruger port=22004>: <Result cmd='df -h' exited=0>,
 <Connection host=130.225.170.33 user=agpbruger port=22005>: <Result cmd='df -h' exited=0>,
 <Connection host=130.225.170.33 user=agpbruger port=22006>: <Result cmd='df -h' exited=0>}

In [9]:
# Run a command as sudo
from invoke import Responder
from fabric import Connection

con = Connection(host= 'agpbruger@130.225.170.33', port = 22022, connect_kwargs= {"password": "MY_SECRET_PASSWORD"})
sudopass = Responder(pattern=r'\[sudo\] password for agpbruger:', response='test\n')
con.run('sudo whoami', pty=True, watchers=[sudopass])

[sudo] password for agpbruger: 
root


<Result cmd='sudo whoami' exited=0>

In [16]:
# Make a connection and run python
from fabric import Connection
con = Connection(host= "agpbruger@c.thomsen-it.dk:22022")
res = con.run('python3 -c "print(\'Hello World\')"',hide=False)
res


Hello World


<Result cmd='python3 -c "print(\'Hello World\')"' exited=0>

In [64]:
# Make a connection and run python that takes 5 min
from fabric import Connection
con = Connection(host= "agpbruger@c.thomsen-it.dk:22022")
res = con.run('python3 -c "import time;time.sleep(60*5)"',hide=False)
res

<Result cmd='python3 -c "import time;time.sleep(60*5)"' exited=0>

In [18]:
# Make a connection and run a python script
from fabric import Connection
con = Connection(host= "agpbruger@c.thomsen-it.dk:22022")
res = con.run('python3 test_program.py "My client name is Paul"',hide=False)
res

Namespace(client_username='My client name is Paul')


<Result cmd='python3 test_program.py "My client name is Paul"' exited=0>

In [58]:
# Run a python script that throws an exception. We should use exit codes instead.
from fabric import Connection
con = Connection(host= "agpbruger@c.thomsen-it.dk:22022")
try:
    res = con.run('python3 -c "raise Exception(\'This is a test\')"',hide=False)
except Exception as e:
    print(e)
    print("----------")
    print(e.result)
    print("----------")
    

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Exception: This is a test


Encountered a bad command exit code!

Command: 'python3 -c "raise Exception(\'This is a test\')"'

Exit code: 1

Stdout: already printed

Stderr: already printed


----------
Command exited with status 1.
(no stdout)
=== stderr ===
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Exception: This is a test

----------


# Running the OnionRTC.py script 

In [85]:

from fabric import Connection

result = None
con = Connection(host= 'localhost')
with con.cd("/home/chris/Documents/OnionRTC-experiment/Selenium"):
    result = con.run('python3 OnionRTC.py', warn=True)

if result is not None:
    if result.failed:
        print(f"Failed with exit code: {result.exited}")
    else:
        print("Success")
else:
    print("No result")

  ___        _             ____ _____ ____ 
 / _ \ _ __ (_) ___  _ __ |  _ \_   _/ ___|
| | | | '_ \| |/ _ \| '_ \| |_) || || |    
| |_| | | | | | (_) | | | |  _ < | || |___ 
 \___/|_| |_|_|\___/|_| |_|_| \_\|_| \____|
                                           

Namespace(client_username='client_username', room_id='room_id', session_length_seconds=60, proxy=False, session_setup_retries=4, client_config='None', headless=True, verbose=False)
2022-12-05 17:08:45 INFO     getUserMedia returned a video track, which is enabled. This means that the webcam works!
2022-12-05 17:08:46 INFO     getUserMedia returned a audio track, which is enabled. This means that the webcam mic works!
2022-12-05 17:08:46 WARNING  Browser settings should be 'True':media.peerconnection.ice.relay_only:True - media.navigator.permission.disabled:True
2022-12-05 17:08:46 INFO     Starting session by navigating to 'https://stage.thomsen-it.dk/#/call/client_username/room_id'
2022-12-05 17:08:48 INFO     We are on the 